# PIC-SURE R API use-case: Phenome-Wide analysis on BioData Catalyst studies

This notebook is an illustration example about how to query data using the R **PIC-SURE API**. It takes as use-case a simple PheWAS analysis. This notebook is intentionally straightforward, and explanation provided are only aimed at guiding through the PheWAS analysis processus. For a more step-by-step introduction to the R PIC-SURE API, see the `R_PICSURE-API_101_PheWAS_example.ipynb` notebook.

**Before running this notebook, please be sure to get an user-specific security token. For more information on how to proceed, see the `get_your_token.ipynb` notebook**

## Environment set-up

### System requirements
- R 3.4 or later

### Packages installation

In [ ]:
source("R_lib/requirements.R")

#### Installing latest R PIC-SURE API libraries from github

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
devtools::install_github("hms-dbmi/pic-sure-r-client", force = T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force = T)

#### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")


In [ ]:
myconnection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [ ]:
resource <- hpds::get.resource(myconnection,
                               resourceUUID = resource_id)

## PheWAS analysis

In a nutshell, this PheWAS analysis follows the subsequent steps:
- Retrieving the variables dictionary, using the PIC-SURE API dedicated methods
- From the info provided by the dictionary, retrieving the data in an exploitable format through PIC-SURE API calls
- Data management
- Running univariate tests again every phenotype variables
- Adjusting for multiple hypotheses testing issue
- Plotting results


This analysis is conducted using individuals enrolled in the COPDGene Study. Overall goal of this cohort is to detect underlying genetic factors to develop Chronic Obstructive Pulmonary Disease (COPD), and currently includes more than 10,000 individuals ([more information on COPDGene Study](http://www.copdgene.org)).

### 1. Retrieving variable dictionary from the Database

Retrieving variables dictionary for COPDGene related variables.

In [ ]:
copdgene_variables <- hpds::find.in.dictionary(resource, "Genetic Epidemiology of COPD (COPDGene)") 
consent_variable <- hpds::find.in.dictionary(resource, "Study Accession with Consent Code")

In [ ]:
copdgene_dic <- lapply(hpds::extract.entries(copdgene_variables), as.character)
consent_dic <- lapply(hpds::extract.entries(consent_variable), as.character) 
plain_variablesDict <- dplyr::bind_rows(copdgene_dic, consent_dic) %>% dplyr::arrange(name)

Here we retrieved information about all the available variables of the COPDGene study, as well as information about the individual consents. The Consent variable will be eventually used to filter the patients pertaining to the COPDGene study. We are then combining both dictionaries into a single one.

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict <- get_multiIndex_variablesDict(plain_variablesDict)
head(variablesDict)

### 2. Selecting variables and retrieving data from HPDS

Subseting to keep only the phenotype variables + the variable "affection status" that will be used as the dependent variable for this illustration use-case.

In [ ]:
# THIS DOES NOT WORK BECAUSE OF PATH NAME CHANGES
mask_copdgene <- variablesDict["level_0"] == 'Genetic Epidemiology of COPD (COPDGene)'
mask_pheno <- variablesDict["level_1"] == 'Subject Phenotype'
mask_status <- variablesDict["level_2"] == 'Affection status'
mask_to_drop <- variablesDict[["simplified_name"]] %in% list("Dbgap_id", "De-identified site code", "A1AD: phenotype/genotype")
mask_vars <- mask_copdgene & (mask_pheno | mask_status) & !mask_to_drop
selected_vars <- variablesDict[mask_vars, ]$name %>% unname() %>% as.list()

In [ ]:
sum(variablesDict["level_2"] == 'Affection status')

In [ ]:
print(selected_vars[1:5])

### Subsetting only COPDGene records

To subset only the subjects from the COPDGene study, we are using the different COPDGene study accession codes, which all begins with 'phs000179...'.

In [ ]:
phs_values = strsplit(consent_dic[["categoryValues"]], ",") %>% unlist()
phs_copdgene = grep("phs000179\\.*", phs_values, perl=T, value=T) %>% as.list()

Retrieving the data:

In [ ]:
my_query = hpds::new.query(resource = resource)

In [ ]:
# by default new queries have consent codes automatically populated, lets clear those values so we can add our own
my_query$filter()$clear()
# adding our own filter criteria
hpds::query.filter.add(query = my_query,
                      keys = consent_dic[["name"]],
                      phs_copdgene)

In [ ]:
hpds::query.select.add(query = my_query,
                      keys = selected_vars)
facts = hpds::query.run(query = my_query, result.type = "dataframe")

In [ ]:
selected_vars

In [ ]:
cat(paste0(dim(facts)[1], " rows, ", dim(facts)[2], " columns"))

In [ ]:
tail(facts)

### 3. Data-management

Since variable names are not the same between the dictionary and the dataframe columns, a workaround is needed: parsing variables names from variables dictionary to match actual dataframe variable names (issue arise from the fact the R data.frame column names cannot contain spaces).

In [ ]:
variablesDict[["df_name"]] <- parsing_varNames(variablesDict[["name"]])

In [ ]:
checking_parsing(names(facts)[-1], variablesDict[mask_vars, ][["df_name"]])

#### Selecting variables regarding their types

One important step in a PheWAS is to get the distinction between categorical and numerical variables. This distinction is straightforward using the variables dictionary.

In [ ]:
mask_categories <- variablesDict[, "categorical"] == TRUE
categorical_varnames <- variablesDict[mask_categories & mask_vars, ][["df_name"]]
continuous_varnames <- variablesDict[!mask_categories & mask_vars, ][["df_name"]]

### Selecting the dependent variable to study
Most of PheWAS studies use a genetic variant as the dependent variable to separate the population between cases and controls. However the population doesn't have to be dichotomized using a genetic variant, and any phenotypic variable could be used to run a PheWAS analysis (see for example [*Neuraz et al.*, 2013](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003405)).

Here we will use the **COPD status** as the case-control variable to dichotomize the population in our analysis, and keep only the population subset containing relevant values for the COPD status (i.e. keeping "Case" and "Control" individuals, thus discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
dependent_varname <- variablesDict[variablesDict[["simplified_name"]] == "Affection status",][["df_name"]]

In [ ]:
categorical_varnames <- categorical_varnames[-which(categorical_varnames == dependent_varname)]

In [ ]:
table(facts[[dependent_varname]])

Then we subset our population regarding the relevant values for the COPD diagnosis variable (i.e. keeping "Case" and "Control" individuals, and discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
mask_dependent_varname = facts[[dependent_varname]] %in% c("Case", "Control")

In [ ]:
facts <- facts[mask_dependent_varname,]

In [ ]:
count_case_control <- table(facts[[dependent_varname]])

In [ ]:
sprintf("Control: %i individuals\nCase: %i individuals",
        count_case_control["Control"],
        count_case_control["Case"]) %>% cat()

### 4. Univariate statistical tests

From this point, each variable present in the `facts_dummies` dataset will be tested again the selected dependent variable, (ie presence or absence of COPD).

Two different association tests will be carried out according to variables data types:
- Mann-Whitney U test for continuous ones
- Fisher exact test for categorical ones

In [ ]:
## Unified univariate tests
### Likelihood ratio test of a logistic model including the independent variable over model without this variable
anova_model <- function(data, dependent_var, independent_var) {
    model <- glm(as.formula(paste(dependent_var, "~ 1 +", independent_var)),
                 data = data,
                 family = binomial(link="logit"))
    model_reduced <- glm(as.formula(paste(dependent_var, "~ 1")),
                         data = data,
                         family = binomial(link="logit"))
    p_val <- anova(model, model_reduced, test =  "LRT")[2, "Pr(>Chi)"]
    return(p_val)
}

In [ ]:
independent_varnames = c(categorical_varnames, continuous_varnames)

In [ ]:
pvalues_list = list()
error_list =  list()
warning_list = list()
cat("length categorical", length(categorical_varnames), "\n")
cat("length independent", length(independent_varnames), "\n")

In [ ]:
for (independent_varname in independent_varnames) {
    dataset <- na.omit(facts[, c(dependent_varname, independent_varname)])
    tryCatch({
                pvalues_list[[independent_varname]] <- anova_model(dataset, dependent_varname, independent_varname)
#                error_list[[independent_varname]] <- NA
 #               warning_list[[independent_varname]] <- NA
    },
             error = function(e) {
                print(paste("error", independent_varname))
                pvalues_list[[independent_varname]] <- NA
                error_list[[independent_varname]] <- e
                warning_list[[independent_varname]] <- NA
             },
             warning = function(w) {
                print(paste("warning", independent_varname))
                pvalues_list[[independent_varname]] <- NA
                error_list[[independent_varname]] <- NA
                warning_list[[independent_varname]] <- w
             }
        )
}

In [ ]:
df_pvalues <- data.frame(
    "df_name" = names(pvalues_list),
    "pvalues" = unlist(unname(pvalues_list)),
    stringsAsFactors = F
)

In [ ]:
df_pvalues[["log_pvalues"]] <- -log10(df_pvalues$pvalues)

In [ ]:
variablesDict_enhanced <- dplyr::left_join(variablesDict, df_pvalues, by="df_name")
variablesDict_enhanced <- dplyr::left_join(variablesDict, df_pvalues, by="df_name")

#### Univariate test p-values distribution

In [ ]:
ggplot(aes_string(x = "pvalues",
                 fill = "categorical"),
       data = variablesDict_enhanced) +
geom_histogram(bins=20, position = "dodge") +
scale_fill_brewer(palette='Paired') +
labs(title = "Distribution of non-adjusted p-values among tested phenotypes ",
    subtitle = expression(italic("Likelihood Ratio Test"))) +
xlab("Unadjusted p-values") +
ylab("Count") +
theme_bw()

## 5. Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple testing problem (increase in the probability of getting statistically significant associations), we will use the Bonferroni correction method. Although many other multiple-comparison corrections exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
variablesDict_enhanced$adj_pvalues <- p.adjust(variablesDict_enhanced$pvalues, method="bonferroni")

In [ ]:
variablesDict_enhanced$log_adj_pvalues <- -log10(variablesDict_enhanced$adj_pvalues)

## 6. Result visualisations: Manhattan plot

Manhattan plot is the classical way to plot the results of a PheWAS analysis. It plots every tested phenotypical variables on the X-axis, against their *-log(pvalue)* on the Y-axis. The horizontal line represents the adjusted significance level threshold.

#### Preparing data

In [ ]:
corrected_alpha <- 0.05/length(variablesDict_enhanced$pvalues) # Using Bonferonni method
adj_corrected_alpha <- -log10(corrected_alpha)

In [ ]:
non_nan <- which(!is.na(variablesDict_enhanced$pvalues))
plot_df <- variablesDict_enhanced[non_nan, ]
plot_df$log_pvalues <- round(plot_df$log_pvalues, 5)

plot_df = variablesDict[, c("df_name", "level_2")] %>%
plyr::rename(replace = c("level_2" = "category")) %>%
right_join(plot_df, by="df_name")

plot_df$df_name <- as.factor(plot_df$df_name)
plot_df <- plot_df[order(plot_df$category),]
plot_df$category <- factor(plot_df$category)
plot_df$name <- factor(plot_df$name, levels=plot_df$name[order(plot_df$category)])

In [ ]:
# Suppressing Inf log(p-values)
plot_df <- plot_df[plot_df$log_pvalues != Inf,]

In [ ]:
# Selecting 4 largest p-values, to be annotated in the Manatthan plot

largest_pvalues_indices <- order(plot_df[["log_pvalues"]], decreasing=T)[1:4]
plot_df$to_annotate <- "no"
plot_df[largest_pvalues_indices, "to_annotate"] <- "yes"

#### Plotting the data

In [ ]:
options(repr.plot.width=12, repr.plot.height=8)

In [ ]:
my_theme <- theme_bw() +
theme(axis.title.y = element_text(face="italic", size=15),
      title = element_text(size=20),
      axis.title.x = element_text(size=15),
      axis.text.x = element_text(angle=35, hjust=1),
      legend.position = "none",
      panel.grid.major.x = element_blank()
      )


In [ ]:
paired_colors <- c("navyblue", "lightskyblue")
#paired_colors <- c("green", "red")

In [ ]:
# Manatthan plot using geom_jitter --> more suitable when number of point per category is enough
ggplot(plot_df, aes(x=category, y=log_pvalues)) +
geom_jitter(alpha=1, aes_string(colour="category"),
           width=0.5,
           size=3) +
geom_hline(yintercept=adj_corrected_alpha, linetype="dashed") +
scale_y_continuous(expand = c(0, 20) ) +
scale_color_manual(values = rep(paired_colors, times=20)) +
geom_label_repel( data=subset(plot_df, to_annotate=="yes"), aes(label=simplified_name), size=3.5) +
labs(title="Association between phenotypes variables and gene mutation (COPD status)",
    x="Phenotypes",
    y="- log10(p-values)",
    colour="Phenotypes categories") +
my_theme



Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not that surprising given the nature of our dependent variable: a lot of the collected phenotypic variables are correlated to the COPD status.

This code can be used directly with any other variable present in the variable Dictionary. Only the `dependent_var_name` variable need to be changed.